In [97]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import itertools
import operator
import collections
import random
import functools
import statistics
import math
import regex
import pprint as pp
import networkx as nx


plt.rc('image', cmap='viridis')
plt.style.use('dark_background')


def Input(day):
    "Open input file for the day"
    file = '2023/input{0}.txt'.format(day)
    return open(file)

def strip_lines(f):
    return [l.strip() for l in f.readlines()] 


## Day 1: Trebuchet?!

In [2]:
with Input(1) as f:
    lines = strip_lines(f)

def calibrate_one(line):
    digits = regex.findall('\d', line)
    return int(digits[0]) * 10 + int(digits[-1])

def calibrate_two(line):
    digits = regex.findall('\d|one|two|three|four|five|six|seven|eight|nine', line, overlapped=True)
    return word_to_int(digits[0]) * 10 + word_to_int(digits[-1])

def word_to_int(word):
    match word:
        case 'one':
            return 1
        case 'two':
            return 2
        case 'three':
            return 3
        case 'four':
            return 4
        case 'five':
            return 5
        case 'six':
            return 6
        case 'seven':
            return 7
        case 'eight':
            return 8
        case 'nine':
            return 9
        case _:
            return int(word)

print('Part One: ', sum(map(calibrate_one, lines)))
print('Part Two: ', sum(map(calibrate_two, lines)))


Part One:  55172
Part Two:  54925


# Day 2: Cube Conundrum

In [115]:
with Input(2) as f:
    lines = strip_lines(f)

def parse_game(line):
    """ 
    Find all the numbers for each color and return a tuple
    of ([red], [green], [blue], game_number)
    """
    game_number = int(regex.match(r'Game (\d+)', line)[1])
    red = list(map(int, regex.findall(r'(\d+) red', line)))
    green = list(map(int, regex.findall(r'(\d+) green', line)))
    blue = list(map(int, regex.findall(r'(\d+) blue', line)))
    game = (red, green, blue, game_number)
    return game


def possible(game):
    #12 red cubes, 13 green cubes, and 14 blue cubes
    red = all([True if red <= 12 else False for red in game[0]])
    green = all([True if green <= 13 else False for green in game[1]])
    blue = all([True if blue <= 14 else False for blue in game[2]])
    return red and green and blue

games = [parse_game(line) for line in lines]

print('Part One: ', sum([game[3] for game in games if possible(game)]))
print('Part Two: ', sum(max(game[0]) * max(game[1]) * max(game[2]) for game in games))

Part One:  2278
Part Two:  67953
